# Import Dataset and Libraries

In [5]:
%pip optuna==3.2.0
%pip install catboost==1.1.1
%pip install pycountry==22.1.10
%pip install scikit-learn==1.2.2
%pip install googletrans==4.0.0-rc1
%pip install imbalanced-learn==0.10.1

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "optuna==3.2.0"



Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 0.1.1
ERROR: Could not find a version that satisfies the requirement catboost==1.1.1 (from versions: 1.2, 1.2.1, 1.2.1.1, 1.2.2)
ERROR: No matching distribution found for catboost==1.1.1

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [182]:
import functools

import joblib
import numpy as np
import optuna
import pandas as pd
import pycountry
from catboost import CatBoostClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score
)
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

TRAIN_FILE = "./train.csv"
TEST_FILE = "./submission.csv"
MODEL_FILE = "./catboost_params.pkl"

DO_OPTIMIZATION = False
RANDOM_STATE = 42

In [183]:
df_train = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE)
df_submission = pd.read_csv(TEST_FILE)

X, y = df_train.drop("is_converted", axis=1), df_train["is_converted"]
X_test = df_test.drop(["is_converted", "id"], axis=1)

# Data Preprocessing

## 1. Data Imputation

In [184]:
values = {
    **{column: 0 for column in X.columns},
    "inquiry_type": "Quotation or Purchase Consultation",
    "customer_country": "Not Found",
}

X.fillna(value=values, inplace=True)
X_test.fillna(value=values, inplace=True)

## 2. Data Cleaning 

### Feature: Expected Timeline ("expected_timeline")

In [185]:
def expected_timeline_remap(line):
    if not isinstance(line, str):
        return np.nan

    line = (
        line.lower()
        .strip()
        .replace("then", "than")
        .replace("_", "")
        .replace("~", "")
        .replace(" ", "")
        .replace("-", "")
    )

    if line in ["lessthan3months", "onemonth"]:
        return 1.5
    elif line in ["3months6months", "lessthan6months", "lessthan5months"]:
        return 4.5
    elif line == "6months9months":
        return 7.5
    elif line == "9months1year":
        return 10.5
    elif line == "morethanayear":
        return 12
    else:
        return np.nan


X["expected_timeline"] = [
    expected_timeline_remap(x) for x in X["expected_timeline"].values
]
X_test["expected_timeline"] = [
    expected_timeline_remap(x) for x in X_test["expected_timeline"].values
]

X["expected_timeline"].fillna(X["expected_timeline"].mean(), inplace=True)
X_test["expected_timeline"].fillna(X["expected_timeline"].mean(), inplace=True)

### Feature: Customer Country ("customer_country")

In [186]:
def preprocess_country(country):
    if not isinstance(country, str):
        return "Not Found"

    country = country.split("/")[-1].strip()
    country = pycountry.countries.get(name=country)
    return "Not Found" if country is None else country.alpha_3


X["customer_country"] = [
    preprocess_country(country) for country in X["customer_country"]
]

X_test["customer_country"] = [
    preprocess_country(country) for country in X_test["customer_country"]
]

In [187]:
def country_to_rate(country_alpha_code):
    count = value_list[country_alpha_code]
    return count / total

value_list = X["customer_country"].value_counts()
total = len(X["customer_country"])
X["customer_country"] = X["customer_country"].apply(country_to_rate)

value_list = X_test["customer_country"].value_counts()
total = len(X_test["customer_country"])
X_test["customer_country"] = X_test["customer_country"].apply(country_to_rate)

### Feature: Business Unit ("business_unit")

In [188]:
X["business_unit"] = X["business_unit"].astype(
    pd.CategoricalDtype(categories=["ID", "AS", "IT", "Solution", "CM"])
)
X_test["business_unit"] = X_test["business_unit"].astype(
    pd.CategoricalDtype(categories=["ID", "AS", "IT", "Solution", "CM"])
)

### Feature: Inquiry Type ("inquiry_type")

In [189]:
inquiry_type_remap = {
    # Quotation: Asking about the product itself.
    "Q": [
        "Quotation or Purchase Consultation",
        "Quotation or purchase consultation",
        "Quotation or Purchase consultation",
        "quotation_or_purchase_consultation",
        "Purchase or Quotation",
        "Purchase",
        "Request for quotation or purchase",
        "Sales Inquiry",
        "sales",
        "quotation_",
    ],
    # Usage: Asking about how to use the product.
    "U": [
        "Usage or Technical Consultation",
        "Usage or technical consultation",
        "usage or technical consultation",
        "usage_or_technical_consultation",
        "Technical Consultation",
        "Technical Support",
        "technical",
        "technical_consultation",
        "Request for technical consulting",
        "Product Information",
        "i want to know the details about it",
        "first Info and pricing",
        "Toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung",
        "tôi cần tham khảo giá và giải pháp từ LG",
        "Trainings",
    ],
    "O": [
        "Other",
        "Others",
        "other_",
        "other",
        "others",
        "Etc.",
        "ETC.",
        "Not specified",
        "(Select ID_Needs)",
        "IDB",
        "Services",
        "Request for Partnership",
        "Request a Demo",
        "Request for Distributorship",
        "Customer Suggestions",
        *(
            df_train["inquiry_type"]
            .value_counts()
            .loc[df_train["inquiry_type"].value_counts() < 10]
            .index.tolist()
        ),
    ],
}

for key, value in inquiry_type_remap.items():
    X.loc[X["inquiry_type"].isin(value), "inquiry_type"] = key
    X_test.loc[X_test["inquiry_type"].isin(value), "inquiry_type"] = key

### Feature: Customer Position ("customer_position")

In [190]:
X['customer_position'] = X['customer_position'].replace('decision maker', 'decision-maker')
X['customer_position'] = X['customer_position'].replace('decision influencer', 'decision-influencer')
X['customer_position'] = X['customer_position'].replace('architect/consultant', 'architecture/consult')
X['customer_position'] = X['customer_position'].replace('physics and mathematics teacher', 'math and physics teacher')
X['customer_position'] = X['customer_position'].replace('vicepresident', 'vice president')
X['customer_position'] = X['customer_position'].replace('others', 'other')

X_test['customer_position'] = X_test['customer_position'].replace('decision influencer', 'decision-influencer')
X_test['customer_position'] = X_test['customer_position'].replace('architect/consultant', 'architecture/consult')
X_test['customer_position'] = X_test['customer_position'].replace('physics and mathematics teacher', 'math and physics teacher')
X_test['customer_position'] = X_test['customer_position'].replace('vicepresident', 'vice president')
X_test['customer_position'] = X_test['customer_position'].replace('others', 'other')

In [191]:
# Classification by position and occupation
C_position_high_executive = [
    'manager', 'ceo/founder', 'c-level executive',
    'vice president', 'c-levelexecutive', 'founder',
    'commercial consultant', 'government',
    'operations', 'other - please specify - cedia association',
    'leadership/executive office/owner', 'market intelligence/research',
    'chairman', 'co-founder', 'chief executive officer',
    'ceo/fundador', 'gerente', 'president', 'genel müdür', 'bulgaria',
]

C_position_low_executive = [
    'director', 'intern', 'trainee', 'entry level',
    'employee', 'administrative', 'manufacturer',
    'system integrator', 'distributor', 'business unit director',
    'business development', 'product management', 'business partner',
    'subsidiary sales (ise)', 'business development/sales',
    'sales',
]

C_position_education = [
    'physics teacher', 'assistant professor',
    'maths lecturer', 'science teacher', 'physics faculty',
    'teacher/middle school coordinator', 'math and physics teacher',
    'professor of mathematics', 'prof.', 'professor of mathematics',
    'academic specialist', 'principal at oxford integrated pu science college'
    'assistant professor of English', 'educator', 'professor',
    'quantitative aptitude faculty', 'english trainer for ielts,toefl,pte,gre,sat exams.'
    'associate professor', 'principal & director', 'hon dean',
    'pgt physics', 'education professional', 'chemistry teacher',
    'director cum faculty at gaining apex coaching centre',
    'academic coordinator/ post graduate teacher (accountancy, business studies)/ tgt (ict)',
    'teacher', 'senior lecturer', 'neet/ olympiad expert faculty',
    'asst prof.', 'associate professor in electronics engg', 'professional trainer',
    'education',
]

C_position_hospital = [
    'hospital', 'medical device manufacturer', 'vp', 'medical imaging specialist',
    'tierarzt', 'főorvos', 'surgery professional', 'pathologist', 'radiology professional'
]

C_position_others = [
    'entry level', 'customer', 'technical',
    'exhibition', 'end-user', 'cargo', 'técnico', 'exhibitiontv',
    'proprietário(a)', 'mindenes', 'research', 'the big boss',
    'partner', 'associate/analyst', 'none', 'unpaid', 'av management',
    'engineering', 'installer', 'homeowner', 'commercial end-user'
    'lider de desarrollo', 'entrepreneurship', 'not applicable',
    'no influence',
]

C_position_consultant = [
    'consultant', 'software/solution provider', 'guest faculty',
    'career coach', 'consulting',
]

In [192]:
position_category= {
    'High Executive': C_position_high_executive,
    'Low Executive': C_position_low_executive,
    'Education': C_position_education,
    'Hospital': C_position_hospital,
    'Consultant': C_position_consultant,
    'Others': C_position_others
}

for category, positions in position_category.items():
    X.loc[X['customer_position'].isin(positions), 'C_position_1'] = category
    X_test.loc[X_test['customer_position'].isin(positions), 'C_position_1'] = category

# 'Hospital', 'Consultant', 'Education'는 무의미한 데이터로 판단 -> Others로 대체
categories_drop = ['Hospital', 'Consultant', 'Education']
X.loc[X['C_position_1'].isin(categories_drop), 'C_position_1'] = 'Others'
X_test.loc[X_test['C_position_1'].isin(categories_drop), 'C_position_1'] = 'Others'

# C_position_category one_hot_encoding
X_encoded = pd.get_dummies(X, columns=['C_position_1'])
X_test_encoded = pd.get_dummies(X_test, columns=['C_position_1'])

In [193]:
col =['C_position_1_High Executive', 'C_position_1_Low Executive', 'C_position_1_Others']

X[col] = X_encoded[col].astype(int)
X_test[col] = X_test_encoded[col].astype(int)

In [194]:
# 기존 열 drop
X.drop(['C_position_1'], axis=1, inplace=True)
X.drop(['customer_position'], axis=1, inplace=True)

X_test.drop(['C_position_1'], axis=1, inplace=True)
X_test.drop(['customer_position'], axis=1, inplace=True)

### Feature: Customer Job ("customer_job")

In [195]:
# engineer category
C_job_engineer = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and 'engineer' in job.lower().replace(" ", "").lower():
        C_job_engineer.append(job)

# tech & IT category
C_job_tech = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and ('tech' in job.lower().replace(" ", "").lower()
                                or 'IT' in job.lower().replace(" ", "").lower()
                                or 'software' in job.lower().replace(" ", "").lower()):
        C_job_tech.append(job)
              
# high_executive category
C_job_high_executive = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and ('admin' in job.lower().replace(" ", "").lower()
                                or 'entrepreneurship' in job.lower().replace(" ", "").lower()
                                or 'executive' in job.lower().replace(" ", "").lower()
                                or 'owner' in job.lower().replace(" ", "").lower() 
                                or 'manage' in job.lower().replace(" ", "").lower()
                                or 'president' in job.lower().replace(" ", "").lower()
                                or 'CEO' in job.lower().replace(" ","").lower()):
        C_job_high_executive.append(job)

# low_executive category
C_job_low_executive = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and ('operations' in job.lower().replace(" ", "").lower() 
                                or 'sale' in job.lower().replace(" ", "").lower()
                                or 'marketing' in job.lower().replace(" ", "").lower()
                                or 'business' in job.lower().replace(" ","").lower()):
        C_job_low_executive.append(job)  

# education category
C_job_education = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and ('education' in job.lower().replace(" ", "").lower() 
                                 or 'professor' in job.lower().replace(" ", "").lower()
                                 or 'educator' in job.lower().replace(" ", "").lower()
                                 or 'teach' in job.lower().replace(" ", "").lower()):
        C_job_education.append(job)

# design category
C_job_design = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and ('art' in job.lower().replace(" ", "").lower() 
                                 or 'design' in job.lower().replace(" ", "").lower()):
        C_job_design.append(job)

# medical category
C_job_medical = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and ('medical' in job.lower().replace(" ", "").lower() 
                                 or 'health' in job.lower().replace(" ", "").lower()
                                 or 'care' in job.lower().replace(" ", "").lower()):
        C_job_medical.append(job)
        
# NaN category
C_job_NaN = []
for job in df_train['customer_job'].unique():
    if pd.isna(job):
        C_job_NaN.append(job)

# Other category
C_job_other = []
for job in df_train['customer_job'].unique():
    if isinstance(job, str) and job not in C_job_engineer + C_job_tech + C_job_high_executive +C_job_medical + C_job_low_executive + C_job_education + C_job_design:
        C_job_other.append(job)


In [196]:
job_category = {
    'Engineer': C_job_engineer,
    'Tech & IT': C_job_tech,
    'high_executive': C_job_high_executive,
    'low_executive': C_job_low_executive,
    'Education': C_job_education,
    'Design': C_job_design,
    'Medical': C_job_medical,
    'NaN': C_job_NaN,
    'Other': C_job_other
}

for category, jobs in job_category.items():
    X.loc[df_train['customer_job'].isin(jobs), 'C_job_category'] = category
    X_test.loc[df_test['customer_job'].isin(jobs), 'C_job_category'] = category

    
# C_job_category one_hot_encoding
X_encoded = pd.get_dummies(X, columns=['C_job_category'])
X_test_encoded = pd.get_dummies(X_test, columns=['C_job_category'])

col =['C_job_category_low_executive', 'C_job_category_high_executive', 'C_job_category_Engineer', 'C_job_category_Education',
     'C_job_category_Tech & IT', 'C_job_category_Design', 'C_job_category_Medical', 'C_job_category_Other', 'C_job_category_NaN']



X[col] = X_encoded[col].astype(int)
X_test[col] = X_test_encoded[col].astype(int)


# 기존 열 drop
X.drop(['C_job_category'], axis=1, inplace=True)
X.drop(['customer_job'], axis=1, inplace=True)

X_test.drop(['C_job_category'], axis=1, inplace=True)
X_test.drop(['customer_job'], axis=1, inplace=True)

# 결측치 처리
X.loc[X['C_job_category_NaN'] == 1, 'C_job_category_Other'] = 1
X.drop(['C_job_category_NaN'], axis=1, inplace=True)
X_test.loc[X_test['C_job_category_NaN'] == 1, 'C_job_category_Other'] = 1
X_test.drop(['C_job_category_NaN'], axis=1, inplace=True)

### Feature: Response Corporate ("response_corporate")

In [197]:
def label_encoding(series: pd.Series) -> pd.Series:
    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

def categorize(num):
    if num > 4000:
        return 'converted_high'
    elif num > 288:
        return 'converted_middle'
    else:
        return 'converted_low'

In [198]:
converted_counts = X.groupby('response_corporate').size().to_dict()
X['response_corporate_num'] = X['response_corporate'].map(converted_counts).fillna(0).astype(int)
X['corporate_converted_category'] = X['response_corporate_num'].apply(categorize)

converted_counts = X_test.groupby('response_corporate').size().to_dict()
X_test['response_corporate_num'] = X_test['response_corporate'].map(converted_counts).fillna(0).astype(int)
X_test['corporate_converted_category'] = X_test['response_corporate_num'].apply(categorize)

X['corporate_category'] = label_encoding(X["corporate_converted_category"])
X_test['corporate_category'] = label_encoding(X_test["corporate_converted_category"])

del X['corporate_converted_category']
del X_test['corporate_converted_category']

### Feature: Product Category ("product_category")

In [199]:
def preprocess_product_dict(df: pd.DataFrame) -> dict:
    product_category_remap = {
        # IS: Integrated Solutions Inquiry - INTEGRATED SOLUTIONS
        "IS" : [
            "monitor signage",
            "commercial tv",
            "monitor",
            "monitor tv",
            "pc",
            "projector",
            "robot",
            "system ac",
            "ems",
            "rac",
            "tv",
            "refrigerator",
            "washing machine",
            "aircare",
            "vaccum cleaner",
            "styler",
            "dryer",
            "built-in/cooking",
            "home beauty",
            "water care",
            "audio/video",
            "lg thinq home",
            "smart home",
            "lg one:quick_flex",    # healthcare solutions
            "one:quick",    # healthcare solutions
            "quick",    # healthcare solutions
            "services",
        ],
        # CD: B2B Product Inquiry - Commercial Display
        "CD": [
            "^(?=.*\d)[A-Za-z0-9.\s-]+$",    # tv panel model name filter
            "medical display",
            "led bloc",
            "uhd signage",
            "commercial display",
            "oled signage",
            "led signage",
            "video wall signage",
            "interactive signage",
            "high brightness signage",
            "special signage",
            "standard signage",
            "hotel tv",
            "hospital tv",
            "accessories",
            "software solution",
            "signage care solution",
            "webos",
            "pro:centric",
            "one:quick series",
            "signage",
            "display",
            'led',
            'wall',    # video wall,
            'MAGNIT',
            "centric",
            "all",
            "idb",
            "virtual",
            "virtual production",
            "createboard",
        ],
        # IT: B2B Product Inquiry - IT PRODUCTS
        "IT": [
            "monitor",
            "laptop",
            "projector",
            "cloud device",
        ],
        # HE: B2B Product Inquiry - HVAC/ESS
        "HE": [
            "control",
            "ventilation",
            "vrf",
            "split",
            "multi-split",
            "single-split",
            "chiller",
            "heating",
            "energy storage system",
            "ess",
            "solar",
            "heat",
            "water",
            "air condition",
            "air",
            "ceiling",
            "cassette",
            "ondition",
            "cool",
            "multi",
            "support",
        ],
        # CM: B2B Product Inquiry - Compressor and Motor
        "CM": [
            "reciprocating compressor",
            "rotary compressor",
            "scroll compressor",
            "compressor",
            "motor",
        ],
        # RB: B2B Product Inquiry - Robot
        "RB": [
            "LG CLOi UV-C Bot",
            "LG CLOi ServeBot",
            "Shelf type",
            "Drawer type",
            "LG CLOi GuideBot",
            "CLOi",
            "UV-C",
            "Bot",
        ],
        # AM: B2B Product Inquiry - ADVANCED MATERIALS
        "AM": [
            "Antimicrobial",
            "Porcelain enamel",
            "Porcelain",
            "enamel",
            "Specialty glass",
            "Specialty",
            "glass",
        ],
        # others
        "others": [
            "[\(\)]",
            "MISSING_VALUE",    # missing value
            "commercial laundry"    # B2B Product Inquiry
            "others",
            "ohters",
            "other",
            "ohter",
            'otros',
            'outros',
            'error',
            "etc",
        ]
    }
    
    return product_category_remap

In [200]:
from googletrans import Translator


def translate_sentence(sentence: str, dest: str = "en") -> str:
    translator = Translator()
    return translator.translate(sentence, src="auto", dest=dest).text


def detect_language(sentence: str) -> str:
    translator = Translator()
    return translator.detect(sentence).lang


def apply_translation(text):
    global already
    if text in already: return already[text]
    
    lang = detect_language(text)
    tran = translate_sentence(text)
    already[text] = tran

    return tran

In [201]:
from tqdm import tqdm
tqdm.pandas()

def preprocess_product_category(df: pd.DataFrame) -> pd.DataFrame:
    # making dictionary depending on LG official website
    product_category_remap = preprocess_product_dict(df)
    
    # filling missing value: "0"
    df_pc = df['product_category'].fillna("MISSING_VALUE", inplace=False)

    # preprocessing by category
    for key, value in product_category_remap.items():
        regex = '|'.join(value)
        
        df_pc.loc[df_pc.str.contains(
            regex,         # chekcing string pattern
            na=False,     # missing value return is boolean
            case=False    # ignore upper and lower
        )] = key
        
    # 'product_category' 열에서 product_category_remap에 없는 값을 필터링
    filtered_categories = df_pc[~df_pc.isin(product_category_remap)]    
    # tqdm을 사용하여 진행 상태를 표시하면서 apply_translation 함수 적용
    filtered_categories = filtered_categories.progress_apply(apply_translation)
    # 기존 df에 번역한 값을 저장
    df_pc.loc[filtered_categories.index] = filtered_categories
    
    # 20개보다 적게 있는 자잘한 값들을 전부 etc에 추가
    etc = (
        df_pc
        .value_counts()
        .loc[df_pc.value_counts() < 20]
        .index.tolist()
    )
    product_category_remap['others'].extend(etc)
        
    # preprocessing by category
    for key, value in product_category_remap.items():
        regex = '|'.join(value)
        
        df_pc.loc[df_pc.str.contains(
            regex,         # chekcing string pattern
            na=False,     # missing value return is boolean
            case=False    # ignore upper and lower
        )] = key 
  
    return df_pc

In [202]:
# translator가 오래 걸려, 저장용 dictionary 선언
already = dict()
df_pc = preprocess_product_category(df_train)
X['product_category'] = df_pc

# translator가 오래 걸려, 저장용 dictionary 선언
already = dict()
df_pc = preprocess_product_category(df_test)
X_test['product_category'] = df_pc

  0%|          | 0/710 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.15it/s]


## 3. Feature Engineering

In [203]:
country_frequency = X["customer_country"].value_counts(normalize=True)
X["customer_country_frequency"] = X["customer_country"].map(country_frequency)
X_test["customer_country_frequency"] = X_test["customer_country"].map(country_frequency)

country_target = (
    pd.concat([X, y], axis=1).groupby("customer_country")["is_converted"].mean()
)
X["customer_country_target"] = X["customer_country"].map(country_target)
X_test["customer_country_target"] = X_test["customer_country"].map(country_target)

owner_frequency = X["lead_owner"].value_counts(normalize=True)
X["lead_owner_frequency"] = X["lead_owner"].map(owner_frequency)
X_test["lead_owner_frequency"] = X_test["lead_owner"].map(owner_frequency)

owner_target = pd.concat([X, y], axis=1).groupby("lead_owner")["is_converted"].mean()
X["lead_owner_target"] = X["lead_owner"].map(owner_target)
X_test["lead_owner_target"] = X_test["lead_owner"].map(owner_target)

business_weight = {
    "ID": 0.064566116,
    "AS": 0.026845638,
    "IT": 0,
    "Solution": 0.034482759,
    "CM": 0,
}

X["com_reg_ver_win_rate_per_bu"] = X["com_reg_ver_win_rate"] * X["business_unit"].map(
    business_weight
)
X_test["com_reg_ver_win_rate_per_bu"] = X_test["com_reg_ver_win_rate"] * X_test[
    "business_unit"
].map(business_weight)

## 4. Feature Encoding

In [204]:
def encode_features(
    df: pd.DataFrame,
    features=[
        "enterprise",
        "business_unit",
        "inquiry_type",
        "product_category",
    ],
) -> pd.DataFrame:
    df_encoded = pd.get_dummies(df[features], columns=features)
    df_encoded = df_encoded.apply(lambda x: x.astype("category").cat.codes)
    df = pd.concat([df, df_encoded], axis=1).drop(features, axis=1)
    return df


X = encode_features(X)
X_test = encode_features(X_test)

## 5. Outlier Removal

In [205]:
Q1 = X["historical_existing_cnt"].quantile(0.25)
Q3 = X["historical_existing_cnt"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = pd.concat([X, pd.DataFrame({"is_converted": y})], axis=1)
df = df[
    (df["historical_existing_cnt"] >= lower_bound)
    & (df["historical_existing_cnt"] <= upper_bound)
]

## 6. Feature Normalization

In [206]:
standard_transformer = Pipeline(steps=[("standardize", StandardScaler())])
column_transformer = ColumnTransformer(
    transformers=[
        (
            "std",
            standard_transformer,
            [
                "lead_desc_length",
                "historical_existing_cnt",
                "bant_submit",
                "expected_timeline",
            ],
        ),
    ],
    verbose_feature_names_out=False,
    remainder="passthrough",
)
column_transformer.set_output(transform="pandas")

X = column_transformer.fit_transform(X)
X_test = column_transformer.transform(X_test)

## 7. Feature Removal

In [207]:
features_to_drop = [
    # "customer_country",
    "customer_country.1",
    "business_subarea",
    "business_area",
    "customer_idx",
    # "product_category",
    "product_subcategory",
    "product_modelname",
    # "customer_position",
    # "customer_job",
    "customer_type",
    "response_corporate",
    "lead_owner",
]

X = X.drop(features_to_drop, axis=1)  # type: ignore
X_test = X_test.drop(features_to_drop, axis=1)  # type: ignore

# 8. Hyperparameter Tuning

In [208]:
def build_catboost_params(trial):
    return {
        "iterations": trial.suggest_int("cat__iterations", 500, 1500),
        "learning_rate": trial.suggest_float("cat__learning_rate", 1e-3, 0.1),
        "depth": trial.suggest_int("cat__depth", 1, 10),
        "subsample": trial.suggest_float("cat__subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("cat__colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("cat__min_data_in_leaf", 1, 100),
        "verbose": False,
        "random_state": RANDOM_STATE,
        "auto_class_weights": "Balanced",
    }


def optimize(X, y, n_trials) -> None:
    def _optimize(trial, X, y):
        model = CatBoostClassifier(**build_catboost_params(trial))
        kfold, scores = StratifiedKFold(n_splits=5), []
        for train_idx, val_idx in kfold.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_val)
            prediction = np.where(prediction == "True", True, False)
            scores.append(f1_score(y_val, prediction))
        return np.mean(scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(
        functools.partial(_optimize, X=X, y=y),  # type: ignore
        n_trials=n_trials,
        n_jobs=-1,
        show_progress_bar=True,
    )
    joblib.dump(study, MODEL_FILE)


if DO_OPTIMIZATION:
    optimize(X, y, n_trials=1)

# 9. Modeling

In [209]:
for col in X.columns:
    print(col)
    print(X[col].head())

param_dict = {
    "cat": {
        "verbose": False,
        "random_state": RANDOM_STATE,
        "auto_class_weights": "Balanced",
    },
}

if DO_OPTIMIZATION:
    optimization_results = joblib.load(MODEL_FILE).best_trial.params
    for key, value in optimization_results.items():
        model, param = [x.strip() for x in key.split("__")]
        param_dict[model][param] = value

model = CatBoostClassifier(**param_dict["cat"])

lead_desc_length
0   -0.130302
1    0.126205
2   -0.175568
3   -0.266100
4    0.133749
Name: lead_desc_length, dtype: float64
historical_existing_cnt
0   -0.199877
1    0.319377
2    6.031173
3   -0.199877
4   -0.199877
Name: historical_existing_cnt, dtype: float64
bant_submit
0    1.27736
1    1.27736
2    1.27736
3    1.27736
4    1.27736
Name: bant_submit, dtype: float64
expected_timeline
0   -0.93126
1   -0.93126
2   -0.93126
3   -0.93126
4   -0.93126
Name: expected_timeline, dtype: float64
customer_country
0    0.043323
1    0.043323
2    0.284136
3    0.284136
4    0.284136
Name: customer_country, dtype: float64
com_reg_ver_win_rate
0    0.066667
1    0.066667
2    0.088889
3    0.088889
4    0.088889
Name: com_reg_ver_win_rate, dtype: float64
id_strategic_ver
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: id_strategic_ver, dtype: float64
it_strategic_ver
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: it_strategic_ver, dtype: float64
idit_strategic_ver
0    0.0
1    0.0
2

# K-Fold Cross Validation

In [210]:
kfolds, scores = (
    StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True),
    [],
)

for fold_idx, (train_idx, val_idx) in enumerate(kfolds.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    model.fit(X_train, y_train)
    prediction = model.predict(X_val)
    prediction = np.where(prediction == "True", True, False)

    print(f"Accuracy (Fold {fold_idx}): {accuracy_score(y_val, prediction)}")
    print(f"Precision (Fold {fold_idx}): {precision_score(y_val, prediction)}")
    print(f"Recall (Fold {fold_idx}): {recall_score(y_val, prediction)}")
    print(f"F1-Score (Fold {fold_idx}): {f1_score(y_val, prediction)}")
    print(
        f"ROC-AUC Score (Fold {fold_idx}): "
        + f"{roc_auc_score(y_val, model.predict_proba(X_val)[:,1])}\n"
    )

    scores.append(f1_score(y_val, prediction))

print(f"Cross-Validation Average F1-Score: {np.mean(scores)}")

Accuracy (Fold 0): 0.9211635750421585
Precision (Fold 0): 0.5103611604499704
Recall (Fold 0): 0.8886597938144329
F1-Score (Fold 0): 0.648364046634073
ROC-AUC Score (Fold 0): 0.9702219476868025

Accuracy (Fold 1): 0.9208263069139966
Precision (Fold 1): 0.5089029293509477
Recall (Fold 1): 0.9134020618556701
F1-Score (Fold 1): 0.6536333456289192
ROC-AUC Score (Fold 1): 0.9733135951833234

Accuracy (Fold 2): 0.9175379426644182
Precision (Fold 2): 0.4977653631284916
Recall (Fold 2): 0.9185567010309278
F1-Score (Fold 2): 0.6456521739130435
ROC-AUC Score (Fold 2): 0.9720947052530933

Accuracy (Fold 3): 0.9230185497470489
Precision (Fold 3): 0.5166569257744009
Recall (Fold 3): 0.911340206185567
F1-Score (Fold 3): 0.659455427079448
ROC-AUC Score (Fold 3): 0.9735882726042052

Accuracy (Fold 4): 0.9213255755122691
Precision (Fold 4): 0.510850439882698
Recall (Fold 4): 0.8979381443298969
F1-Score (Fold 4): 0.6512149532710281
ROC-AUC Score (Fold 4): 0.9685812221356462

Cross-Validation Average F1-S

# Create Submission

In [212]:
model.fit(X, y)

prediction = model.predict(X_test)
prediction = np.where(prediction == "True", True, False)

df_submission["is_converted"] = prediction
df_submission.to_csv("submission.csv", index=False)
print(df_submission["is_converted"].value_counts())
df_submission.head()

is_converted
False    4732
True      539
Name: count, dtype: int64


,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,False
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False
